In [4]:
%cd ../app

D:\Programs\workdir\video_membership_tutorial\app


In [5]:
import db
from cassandra.cqlengine.management import sync_table
from users.models import User

In [6]:
db.get_session()
sync_table(User)

In [20]:
post_data = {
    "email": "abc@abc.com",
    "password": "abc123"
}

In [26]:
from pydantic import BaseModel, EmailStr, SecretStr, validator

In [22]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr

In [23]:
UserLoginSchema(**post_data)

UserLoginSchema(email='abc@abc.com', password=SecretStr('**********'))

In [48]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirm: SecretStr
    
    @validator('email')
    def email_available(cls, v, values, **kwargs):
        q = User.objects.filter(email=v)
        
        if q.count() != 0:
            raise ValueError('Email not available')
        
        return v
    
    @validator('password_confirm')
    def password_match(cls, v, values, **kwargs):
        password = values.get('password')
        password_confirm = v
        
        if password != password_confirm:
            raise ValueError('Passwords do not match')
        
        return v

In [49]:
post_data = {
    "email": "abc@abc.com",
    "password": "abc123",
    "password_confirm": "abc123"
}

In [50]:
data = UserSignupSchema(**post_data)

In [51]:
data.dict()['password'].get_secret_value()

'abc123'